In [4]:
import tifffile as tf
import numpy as np
from skimage.feature import peak_local_max
from skimage.filters import gaussian
from matplotlib import pyplot as plt

In [69]:
tif = tf.TiffFile('/home/mjsf3/TrapAnalysis/260522_CecB-  PCPG vesicles- after flushing_1_MMStack_Pos31.ome-1.tif')

frames = tif.asarray()

subtracted_frames = frames[:-1] - frames[1:]


interficiendum = None
for frame_index in range(subtracted_frames.shape[0]):
    frame = subtracted_frames[frame_index]
    
    frame = gaussian(frame,10)
    pos_peaks = peak_local_max(frame,min_distance=40,threshold_rel=0.95)
    neg_peaks = peak_local_max(np.max(frame)-frame,min_distance=40,threshold_rel=0.95)
    
    pos_peaks = np.array(pos_peaks)
    neg_peaks = np.array(neg_peaks)
    
    
    
    peaks = np.vstack((pos_peaks,neg_peaks))
    
    frame  = np.hstack((frame,0-frame))
    if len(pos_peaks) >0 and len(neg_peaks) > 0:
        
        #if we find a positive and a negative peak in the subtracted frame, we believe that a vesicle has moved
        #remains to pair the negative peak and positive peak to make sure a vesicle moved and didnt just burst
        
        pair_vectors = pos_peaks[np.newaxis] - neg_peaks
        print(pair_vectors.shape)
        if pair_vectors.shape[1] ==1:
            if interficiendum is None:
                interficiendum = neg_peaks
                print(neg_peaks)
                print(interficiendum)
            else:
                interficiendum = np.vstack((interficiendum,neg_peaks))
                
        else:
            pair = pair_vectors[np.absolute(pair_vectors[:,:,0]) < 60]
            print(pair_vectors[:,:,0])
            
            if pair_vectors.shape[1] == 1:
                if interficiendum is None:
                    interficiendum = neg_peaks[pair_vectors[:,0] < 60]
                    
                else:
                    interficiendum = np.vstack(interficiendum,neg_peaks[pair_vectors[:,0] < 60])  
                    
            else:
                pair = pair_vectors[pair_vectors[:,:,1] < 0]
                pair = pair[pair[:,0] < 60]
                
                
                if len(neg_peaks) > 1 and len(pair[0]) > 0:
                    print(pair_vectors[:,:,0])
                    print([np.absolute(pair_vectors[:,:,0]) == np.min(np.absolute(pair_vectors[:,:,0]))][0])
                    peak = neg_peaks[[np.absolute(pair_vectors[:,:,0]) == np.min(np.absolute(pair_vectors[:,:,0]))][0][0]]
                    
                else:
                    peak = neg_peaks                    
                
                    
                if interficiendum is None:
                    interficiendum = peak
                else:
                    interficiendum = np.vstack((interficiendum,peak))
                        



(1, 1, 2)
[[289 179]]
[[289 179]]
(1, 1, 2)
(1, 2, 2)
[[  0 -16]]
(1, 1, 2)
(1, 1, 2)
(1, 2, 2)
[[-35   3]]
[[-35   3]]
[[False  True]]
(1, 1, 2)
(1, 1, 2)
(1, 1, 2)
(1, 1, 2)
(1, 1, 2)
(1, 1, 2)
(1, 2, 2)
[[52 62]]
[[52 62]]
[[ True False]]
(1, 1, 2)
(1, 1, 2)
(1, 1, 2)


In [50]:
print(pair_vectors.shape)
print(interficiendum)

(1, 1, 2)
[[289 179]
 [221 441]
 [159 348]
 [306 327]
 [191 207]
 [319 321]
 [377 143]
 [250 397]
 [222 183]
 [254 223]
 [317 193]
 [327 127]
 [312  77]]


In [5]:
subtracted_frames.shape

(1936, 512, 512)

In [3]:
tifw = tf.TiffWriter('./subtracted_frames.tif',imagej=True)
metadata = {}
metadata['frames'] = subtracted_frames.shape[0]

counter = 0
for frame in subtracted_frames:

        

    
    if counter == 0:
        tifw.save(frame,metadata=metadata)
      
    else:
        tifw.save(frame)
    counter +=1
    

In [2]:
tif = tf.TiffFile('./subtracted_frames.tif')

In [21]:
images = tif.asarray()

In [22]:
images

array([[[   11,    20, 65515, ..., 65497,    12,    21],
        [65451, 65447, 65447, ...,    27, 65447, 65462],
        [   49,    24,    58, ...,    48, 65451,   127],
        ...,
        [   58, 65446, 65501, ..., 65441, 65504,   208],
        [65484, 65514,    40, ..., 65510,    96,    21],
        [  133, 65454,     1, ..., 65523, 65469, 65416]],

       [[65421, 65448,     2, ...,     3, 65510, 65519],
        [    2,    22, 65453, ...,    37,   103,    48],
        [65480, 65482, 65494, ..., 65504,    20, 65389],
        ...,
        [65485,    11, 65495, ..., 65517, 65443, 65483],
        [   57,    29, 65531, ..., 65530, 65478, 65421],
        [   25,    93,    62, ..., 65496,   111,    90]],

       [[   62,    43,    84, ..., 65525,     3, 65430],
        [65525,    79,   194, ..., 65438, 65523, 65455],
        [65402,     9, 65435, ...,    11,    18,    38],
        ...,
        [   54,    23,    71, ...,    98,    31,    13],
        [65481, 65409, 65506, ..., 65493, 654

In [11]:
tif.asarray().shape

(512, 512)

In [ ]:
np.savetxt(X=images,f)